In [1]:
k = 1000
model = 'gpt-4o-mini'

In [2]:
%load_ext autoreload
%autoreload 2

import asyncio
import pandas as pd
from transcript_analyzer import TranscriptAnalyzer
from typing import List

from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

from vexa import VexaAPI
from core import system_msg, user_msg,generic_call_stream
from datetime import datetime

from prompts import Prompts
prompts = Prompts()

analyzer = TranscriptAnalyzer()
analyzer.count_documents()

/home/dima/anaconda3/envs/langchain/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1236

In [3]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer, CrossEncoder
from core import count_tokens

In [4]:

from pydantic import BaseModel, Field
from typing import Optional
from core import BaseCall
from datetime import datetime


class Query(BaseCall):
    """make specifc vector_search_queries based on general context"""
    start: Optional[datetime] = Field(None, description="Start time")
    end: Optional[datetime] = Field(None, description="End time")
    vector_search_query: Optional[str] = Field(..., description="""specific search query that contained in the generalcontext directly""")
    
class QueryPlan(BaseCall):
    queries: List[Query] = Field(..., description="List of requests, aim for at least 2 queries, all aparms optional")



In [5]:
summaries = analyzer.get_summaries()

In [6]:
self = analyzer

In [100]:
def build_context_string(summaries, points_by_meeting=None, only_summaries=False, include_all_summaries=True):
    context = []
    meeting_counter = 1
    meeting_ids = []
    for summary in summaries:
        meeting_id = summary['meeting_session_id']
        if include_all_summaries or (points_by_meeting and meeting_id in points_by_meeting):
            start_datetime = pd.to_datetime(datetime.fromtimestamp(summary['start_datetime']))
            context.append(f"# [{meeting_counter}] {summary['meeting_name']} ({start_datetime.strftime('%Y-%m-%d %H:%M')}):\n{summary['content']}")
            
            if not only_summaries and points_by_meeting and meeting_id in points_by_meeting:
                context.append("## Related Quotes:")
                for point in points_by_meeting[meeting_id]:
                    context.append(f"{point['qoutes']}")
            
            context.append("\n---\n")
            meeting_ids.append(meeting_id)
            meeting_counter += 1

    # Join context
    full_context = "\n".join(context)
    return full_context, meeting_ids

In [101]:
query = """find all meetings that where about seo"""

In [102]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S (%A)")

In [103]:
planner_prompt = f"""think step by step of which of the following meetings are relevant to the user request. and create many (5+) requests to the search system which will be used to find similar meetings to the user request, 
                      requests should have at least 125 char each and at least 2 sentences."""
plan = await QueryPlan.call([system_msg(f"""{planner_prompt}.
                                           Now is {now}.
                                           Write queries based on user request and general context you have, don't use start and end if timingis not obvious from the user query.
                                        """), 
                             user_msg(f'general context: last 100 meeting summaries in ascending order: {build_context_string(summaries)}'),
                             
                             user_msg(f'user request: {query}.')])


In [107]:
async def build_context(analyzer, queries, summaries, only_summaries=False,k=20,include_all_summaries=True):
    points_with_scores = await analyzer.multi_search_documents(queries=queries, k=k)
    points_by_meeting = {}
    for point, score, id in points_with_scores:
        if score > 0.1:
            meeting_id = point['meeting_session_id']
            if meeting_id not in points_by_meeting:
                points_by_meeting[meeting_id] = []
            points_by_meeting[meeting_id].append(point)

    

    full_context, meeting_ids = build_context_string(summaries, points_by_meeting, only_summaries,include_all_summaries)
    return full_context, meeting_ids


In [108]:
queries = plan[0].model_dump()['queries']
queries

[{'start': None,
  'end': None,
  'vector_search_query': 'SEO strategies and content generation'},
 {'start': None, 'end': None, 'vector_search_query': 'SEO optimization'},
 {'start': None, 'end': None, 'vector_search_query': 'SEO tool development'},
 {'start': None,
  'end': None,
  'vector_search_query': 'SEO marketing strategies'},
 {'start': None, 'end': None, 'vector_search_query': 'SEO content creation'},
 {'start': None, 'end': None, 'vector_search_query': 'SEO performance review'}]

In [109]:
len(summaries)

195

In [110]:
full_context, meeting_ids = await build_context(analyzer, queries, summaries,include_all_summaries=False)
print(full_context)

Total results before deduplication: 120
Total results after deduplication: 51
# [1] Marketing Strategy Discussion (2024-08-27 22:34):
In the meeting held on August 27, 2024, Olga Nemirovskaya and Dmitriy Grankin discussed marketing strategies, including the integration of Google Analytics with their CRM, launching a Google Ads campaign, and exploring TikTok for audience engagement in Brazil. They emphasized the need for effective content creation and optimization, as well as analyzing competitors' strategies to enhance their marketing efforts.
## Related Quotes:
Dmitriy Grankin:  А что у нас визуалка к тому посту, который я попросил? Dmitriy Grankin:  А я тебе закинула, ты не видел? Olga Nemirovskaya:  Да, закинул. Olga Nemirovskaya:  Висит она в продукте. Olga Nemirovskaya:  А, да, точно. Dmitriy Grankin:  Норм? Dmitriy Grankin:  Зачем так делать-то, я не понимаю. Dmitriy Grankin:  А что ты хочешь? Olga Nemirovskaya:  Ты конкретизируй, как именно ты это видишь. Dmitriy Grankin:  Я же 

In [112]:
messages=[
    system_msg(prompts.perplexity),
    user_msg(f"Based on the following context, answer the question: {query}\n\nContext:\n{full_context}")
]

In [113]:
count_tokens(full_context)

36324

In [114]:
await generic_call_stream(messages)

The meetings specifically focused on SEO are as follows:

1. **SEO Marketing Strategy Discussion (2024-09-06 17:32)**: In this meeting, Olga Nemirovskaya and Dmitriy Grankin discussed competitor SEO marketing strategies and the importance of interpreting data from SimilarWeb to inform their marketing decisions[4].

2. **SEO Tool Development Discussion (2024-10-01 12:19)**: Rick Tousseyn and Dmitriy Grankin explored a content creation tool for SEO optimization, discussing keyword targeting and strategies for improving article rankings. They emphasized the importance of effective content strategies in a competitive market[8].

3. **SEO Strategies and Content Generation Discussion (2024-10-03 12:00)**: Olga Mykhoparkina and Dmitriy Grankin discussed strategies for improving SEO and content generation, including collaboration with an SEO expert to update outdated articles and the potential of AI in content creation[9].

These meetings highlight the ongoing discussions and strategies relate

In [117]:
meeting_ids[8-1]

'bdc05628-4278-4cda-9395-49906c348d7d'

In [121]:

meeting_id_ref = 8
dashboard_url = f"https://dashboard.vexa.ai/#{meeting_ids[meeting_id_ref-1]}"
dashboard_url


'https://dashboard.vexa.ai/#bdc05628-4278-4cda-9395-49906c348d7d'